In [ ]:
import datetime
start_time = datetime.datetime.now().strftime("%d %b, %H:%M:%S")
start_time

'18 Jan, 06:06:04'

In [ ]:
# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 100000  # 200000 to improve

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'mobilenet_tpu': {
        'model_name': 'ssdlite_mobilenet_edgetpu_coco_quant',
        'pipeline_file': 'ssdlite_mobilenet_edgetpu_320x320_coco_quant.config',
        'batch_size': 32
    },
    'mobilenet': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 32
    },
    'mobiledet_tpu': {
        'model_name': 'ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19',
        'pipeline_file': 'ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config',
        'batch_size': 32
    },
    'mobiledet': {
        'model_name': 'ssdlite_mobiledet_cpu_320x320_coco_2020_05_19',
        'pipeline_file': 'ssdlite_mobiledet_cpu_320x320_coco_sync_4x4.config',
        'batch_size': 32
    },    
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'mobiledet'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

print("Selected model:", MODEL)

Selected model: ssdlite_mobiledet_cpu_320x320_coco_2020_05_19


In [ ]:
# use TF 1.x for Object Detection APIs as they are not ported to TF 2.0 yet
%tensorflow_version 1.x

TensorFlow 1.x selected.


## Install required packages

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!pip install tf_slim

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install lvis

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

/content
     |████████████████████████████████| 358kB 13.3MB/s 
Selecting previously unselected package python-bs4.
(Reading database ... 145483 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unp

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

%cd /content

drive_id = '1HQfrA9eHaIrqohTtNpSu8vCSLaZNpZbn'  # Hardhat
# drive_id = '1a8CavWcehN_aYGOrSS8VM4iC_VEkyW2P'  # Scratch

gdd.download_file_from_google_drive(file_id=drive_id,
                                    dest_path='./dataset.zip',
                                    unzip=True)

!rm dataset.zip

!mv data_tfrecord data

/content
Unzipping...Done.


In [ ]:
# !curl -L "https://public.roboflow.com/ds/ub0NLZ3ABo?key=G7EycgLLK6" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
# %cd /content/

In [ ]:
# NOTE: Update these TFRecord names from "cells" and "cells_label_map" to your files!
test_record_fname = '/content/data/data_test.tfrecord'
valid_record_fname = '/content/data/data_valid.tfrecord'
train_record_fname = '/content/data/data_train.tfrecord'
label_map_pbtxt_fname = '/content/data/label_map.pbtxt'

## Download base model

In [ ]:
import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/pretrained_model
total 121M
drwxr-x--- 2 475825 89939 4.0K May 19  2020 .
drwxr-xr-x 1 root   root  4.0K Jan 18 06:07 ..
-rw-r----- 1 475825 89939  32M May 19  2020 model.ckpt-400000.data-00000-of-00001
-rw-r----- 1 475825 89939  14K May 19  2020 model.ckpt-400000.index
-rw-r----- 1 475825 89939 9.9M May 19  2020 model.ckpt-400000.meta
-rw-r----- 1 475825 89939  16M May 19  2020 model.tflite
-rw-r----- 1 475825 89939 4.6K May 19  2020 pipeline.config
-rw-r----- 1 475825 89939  17M May 19  2020 tflite_graph.pb
-rw-r----- 1 475825 89939  47M May 19  2020 tflite_graph.pbtxt


In [ ]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/pretrained_model/model.ckpt'

## Configuring a Training Pipeline

In [ ]:
%cd /content/models/research

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

num_classes = get_num_classes(label_map_pbtxt_fname)

%cd /content/

/content/models/research
/content


In [ ]:
# Edit Pipeline config to load in our new tfrecord that we just created and add quantization aware training.
import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

pipeline = pipeline_pb2.TrainEvalPipelineConfig()                                                                                                                                                                                                          
config_path = '/content/models/research/object_detection/samples/configs/' + pipeline_file
with tf.gfile.GFile(config_path, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline)

pipeline.train_input_reader.tf_record_input_reader.input_path[:] = [train_record_fname]
pipeline.train_input_reader.label_map_path = label_map_pbtxt_fname
pipeline.eval_input_reader[0].tf_record_input_reader.input_path[:] = [valid_record_fname]
pipeline.eval_input_reader[0].label_map_path = label_map_pbtxt_fname
pipeline.train_config.fine_tune_checkpoint = fine_tune_checkpoint
pipeline.train_config.batch_size = batch_size  # Smaller batch size on free gpu to avoid OOM Killer
pipeline.train_config.num_steps = num_steps  # Less steps with free gpu but 10k should be good enough
pipeline.model.ssd.num_classes = num_classes
pipeline.eval_config.metrics_set[:] = ['coco_detection_metrics']
pipeline.eval_config.use_moving_averages = False
# pipeline.eval_config.ClearField("max_evals")


# Quantization Aware Training
# pipeline.graph_rewriter.quantization.delay = 0
# pipeline.graph_rewriter.quantization.weight_bits = 8
# pipeline.graph_rewriter.quantization.activation_bits = 8

config_text = text_format.MessageToString(pipeline)                                                                                                                                                                                                        
with tf.gfile.Open(config_path, "wb") as f:                                                                                                                                                                                                                       
    f.write(config_text)

# This is out config after modifying.
!cat $config_path

model {
  ssd {
    num_classes: 3
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobiledet_cpu"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.03
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.97
          center: true
          scale: true
          epsilon: 0.001
          train: true
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: false
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatc

In [ ]:
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

## Train the model

In [ ]:
print(config_path)
print(num_steps)

/content/models/research/object_detection/samples/configs/ssdlite_mobiledet_cpu_320x320_coco_sync_4x4.config
100000


In [ ]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

Streaming output truncated to the last 5000 lines.
2021-01-18 16:38:04.145177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-18 16:38:04.145525: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1325] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 15216 MB memory) -> physical GPU (device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0)
INFO:tensorflow:Restoring parameters from training/model.ckpt-63685
I0118 16:38:04.146673 139857048897408 saver.py:1284] Restoring parameters from training/model.ckpt-63685
INFO:tensorflow:Running local_init_op.
I0118 16:38:05.446807 139857048897408 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0118 16:38:05.617647 139857048897408 session_manager.py:502] Done running local_init_op.
INFO:tensorf

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir {model_dir}

In [ ]:
end_time = datetime.datetime.now().strftime("%d %b, %H:%M:%S")
print(start_time)
print(end_time)

18 Jan, 06:06:04
18 Jan, 22:40:58


## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={config_path} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

## Test accuracy on test images

In [ ]:
pipeline = pipeline_pb2.TrainEvalPipelineConfig()                                                                                                                                                                                                          
config_path = '/content/models/research/object_detection/samples/configs/' + pipeline_file
with tf.gfile.GFile(config_path, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline)

pipeline.eval_input_reader[0].tf_record_input_reader.input_path[:] = [test_record_fname]
pipeline.train_config.batch_size = 1

config_text = text_format.MessageToString(pipeline)                                                                                                                                                                                                        
with tf.gfile.Open(config_path, "wb") as f:                                                                                                                                                                                                                       
    f.write(config_text)

# This is out config after modifying.
!cat $config_path

model {
  ssd {
    num_classes: 3
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobiledet_cpu"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.03
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.97
          center: true
          scale: true
          epsilon: 0.001
          train: true
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: false
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatc

In [ ]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={config_path} \
    --checkpoint_dir=/content/fine_tuned_model/ \
    --alsologtostderr \
    --run_once=True

W0118 22:41:59.263336 140411789715328 model_lib.py:793] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: None
I0118 22:41:59.263543 140411789715328 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0118 22:41:59.263627 140411789715328 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0118 22:41:59.263708 140411789715328 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0118 22:41:59.263787 140411789715328 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0118 22:41:59.263899 140411789715328 model_lib.py:809] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_t

## Run inference test
Test with images in repository `tensorflow-object-detection/test` directory.

**To test with your own images, you need to place your images inside the `test` directory in this Colab notebook!** More on this below.

In [ ]:
%cd /content/
drive_id = '11nAnbp2Rnb9y1p_FjrkKBE4FWDAKtqh5'  # Hardhat
# drive_id = '15XRe167DzTYb7S6fNDAVCrnIr05GJ6jf'  # Scratch

gdd.download_file_from_google_drive(file_id=drive_id,
                                    dest_path='./dataset.zip',
                                    unzip=True)
!rm dataset.zip

/content
Unzipping...Done.


In [ ]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  '/content/hardhat_test_images/'

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.jpg*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
TEST_IMAGE_PATHS = sorted(TEST_IMAGE_PATHS)
print(TEST_IMAGE_PATHS)

['/content/hardhat_test_images/000007_jpg.rf.8e98fb789dfc011fb526ce49daf8da31.jpg', '/content/hardhat_test_images/000024_jpg.rf.f4dbede31bcaa30e4264e464b2d32266.jpg', '/content/hardhat_test_images/000035_jpg.rf.a3fd81d1d38c11263dc0f6670b50deda.jpg', '/content/hardhat_test_images/000038_jpg.rf.ee7b22dab5abf8252225253d5b7b50cf.jpg', '/content/hardhat_test_images/000045_jpg.rf.f8429e05f94a05b82b3bf7256063c313.jpg', '/content/hardhat_test_images/000052_jpg.rf.629cc4e04ec1578c1e813c84be098fd0.jpg', '/content/hardhat_test_images/000060_jpg.rf.c77953cd46240aa6676f74bbd2cef120.jpg', '/content/hardhat_test_images/000069_jpg.rf.09b9ad6fbe4cdfbe1d38a7d7424868a5.jpg', '/content/hardhat_test_images/000072_jpg.rf.a6086d0ee280ad45b27f1dfb38f08d1f.jpg', '/content/hardhat_test_images/000076_jpg.rf.6e17a0977fc8ac32c1559e1782d550cf.jpg', '/content/hardhat_test_images/000099_jpg.rf.73261a5bc27bf65613d5c6d1596daec5.jpg', '/content/hardhat_test_images/000102_jpg.rf.a547296ebcc407e4c961aa32c85d99a0.jpg', '/c

In [ ]:
%cd /content/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

/content/models/research/object_detection


In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [ ]:
# Output images not showing? Run this cell again, and try the cell above
# This is needed to display the images.
%matplotlib inline

In [ ]:
len(TEST_IMAGE_PATHS)

1055

In [ ]:
from tqdm import tqdm

for image_path in tqdm(TEST_IMAGE_PATHS[:100]):
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=4)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    save_name = '.'.join(image_path.split('.')[:-1]) + '_prediction.jpg'
    plt.savefig(save_name, bbox_inches='tight')

In [ ]:
from google.colab import files

!zip -r /content/mobiledet.zip /content/fine_tuned_model/
# files.download("/content/mobiledet.zip")

updating: content/fine_tuned_model/ (stored 0%)
updating: content/fine_tuned_model/model.ckpt.index (deflated 68%)
updating: content/fine_tuned_model/saved_model/ (stored 0%)
updating: content/fine_tuned_model/saved_model/saved_model.pb (deflated 10%)
updating: content/fine_tuned_model/saved_model/variables/ (stored 0%)
updating: content/fine_tuned_model/frozen_inference_graph.pb (deflated 9%)
updating: content/fine_tuned_model/model.ckpt.meta (deflated 93%)
updating: content/fine_tuned_model/pipeline.config (deflated 69%)
updating: content/fine_tuned_model/checkpoint (deflated 42%)
updating: content/fine_tuned_model/model.ckpt.data-00000-of-00001 (deflated 5%)


In [ ]:
#take a look at the kind of GPU we have
!nvidia-smi

Mon Jan 18 22:48:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    33W / 250W |   1571MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Download inferred images

!mkdir /content/result_images
!cp /content/hardhat_test_images/*_prediction.jpg /content/result_images/

!zip -r /content/mobiledet_img_results.zip /content/result_images
# files.download("/content/mobiledet_img_results.zip")

  adding: content/result_images/ (stored 0%)
  adding: content/result_images/000270_jpg.rf.28434781eaeebf0feab466eed579e8d3_prediction.jpg (deflated 1%)
  adding: content/result_images/000304_jpg.rf.b43923673e0b7f4148ca29369557aa42_prediction.jpg (deflated 2%)
  adding: content/result_images/000038_jpg.rf.ee7b22dab5abf8252225253d5b7b50cf_prediction.jpg (deflated 1%)
  adding: content/result_images/000007_jpg.rf.8e98fb789dfc011fb526ce49daf8da31_prediction.jpg (deflated 1%)
  adding: content/result_images/000358_jpg.rf.28808a9f7792bcec6621535b75826490_prediction.jpg (deflated 1%)
  adding: content/result_images/000152_jpg.rf.f9c2a41a8714cffa2a600796c692024a_prediction.jpg (deflated 1%)
  adding: content/result_images/000052_jpg.rf.629cc4e04ec1578c1e813c84be098fd0_prediction.jpg (deflated 1%)
  adding: content/result_images/000642_jpg.rf.cc772ca97cdfe9b83b116aa1f2328dd5_prediction.jpg (deflated 1%)
  adding: content/result_images/000151_jpg.rf.8f93d9e83e73fe3601ecdc1ca55e9960_prediction.j